## Imports

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
from fastai.tabular import *
from sklearn.model_selection import StratifiedKFold

## Paths

In [ ]:
PATH = Config.data_path()/'competitions/elo-merchant-category-recommendation/'

In [ ]:
train_df = pd.read_pickle(PATH/'joined_nn_1')

In [ ]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
feature_1,201917.0,3.105311e+00,1.186160e+00,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,5.000000e+00
feature_2,201917.0,1.745410e+00,7.513617e-01,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00
feature_3,201917.0,5.655690e-01,4.956833e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
target,201917.0,-3.936363e-01,3.850500e+00,-3.321928e+01,-8.831102e-01,-2.343689e-02,7.654530e-01,1.796507e+01
new_transactions_count,179986.0,6.776555e+00,6.694579e+00,1.000000e+00,2.000000e+00,5.000000e+00,9.000000e+00,1.090000e+02
new_authorized_flag_sum,179986.0,6.776555e+00,6.694579e+00,1.000000e+00,2.000000e+00,5.000000e+00,9.000000e+00,1.090000e+02
new_authorized_flag_mean,179986.0,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
new_merchant_id_nunique,179986.0,6.773677e+00,6.691423e+00,1.000000e+00,2.000000e+00,5.000000e+00,9.000000e+00,1.090000e+02
new_city_id_nunique,179986.0,2.309430e+00,1.670109e+00,1.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,2.800000e+01
new_purchase_amount_sum,179986.0,-3.741887e+00,4.395309e+00,-7.081349e+01,-5.257329e+00,-2.686252e+00,-1.211768e+00,1.090862e+02


In [ ]:
n = len(train_df); n

201917

## Data pipeline

In [ ]:
train_df = pd.read_pickle(PATH/'joined_nn_1')
test_df = pd.read_pickle(PATH/'joined_test_nn_1')

In [ ]:
train_df['outliers'] = 0
train_df.loc[train_df['target']<-30, 'outliers'] = 1

In [ ]:
train_df.shape, test_df.shape

((201917, 53), (123623, 52))

In [ ]:
procs=[FillMissing, Categorify, Normalize]

In [ ]:
dep_var = 'target'
exclude = ['first_active_month', 'card_id', 'outliers']
cont_vars,cat_vars = cont_cat_split(train_df, max_card=20, dep_var=dep_var)

In [ ]:
cont_vars = list(set(cont_vars)-set(exclude))
cat_vars  = list(set(cat_vars)-set(exclude))

In [ ]:
strat = StratifiedKFold(n_splits=5, shuffle=True, random_state=15)
folds = list(strat.split(train_df, train_df.outliers))

In [ ]:
#db0 = TabularList.from_df(train_df, path=PATH, cat_names=cat_vars, cont_names=cont_vars, procs=procs); db0;

In [ ]:
#db1 = db0.split_by_idx(folds[0][1]); db1;

In [ ]:
#db2 = db1.label_from_df(cols=dep_var, label_cls=FloatList)

In [ ]:
#data = db2.databunch()

In [ ]:
data = (TabularList.from_df(train_df, path=PATH, cat_names=cat_vars, cont_names=cont_vars, procs=procs)
                   .split_by_idx(folds[0][1])
                   .label_from_df(cols=dep_var, label_cls=FloatList)
                   .databunch())

In [ ]:
cat_sz = [(c, len(train[c].cat.categories)+1) for c in cat_vars]
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]
y_range=(np.min(y),np.max(y))

In [ ]:
cat_sz = [(c, len(train_df[c].cat.categories)+1) for c in cat_vars]

In [ ]:
cat_vars

['first_active_monthIs_year_end',
 'first_active_monthIs_quarter_end',
 'first_active_monthIs_month_end',
 'first_active_monthIs_quarter_start',
 'first_active_monthIs_year_start',
 'first_active_monthIs_month_start',
 'new_authorized_flag_mean_na',
 'new_installments_min_na',
 'new_installments_sum_na',
 'new_purchase_amount_max_na',
 'new_city_id_nunique_na',
 'new_transactions_count_na',
 'new_month_lag_min_na',
 'new_installments_median_na',
 'old_authorized_flag_mean_na',
 'new_month_lag_max_na',
 'new_merchant_id_nunique_na',
 'new_authorized_flag_sum_na',
 'new_purchase_amount_median_na',
 'new_purchase_amount_std_na',
 'new_installments_std_na',
 'new_installments_max_na',
 'new_purchase_amount_min_na',
 'new_purchase_amount_sum_na']

In [ ]:
train_df['new_transactions_count_na']

## Model

In [ ]:
max_log_y = np.log(np.max(train_df['Sales'])*1.2)
y_range = torch.tensor([0, max_log_y], device=defaults.device)

In [ ]:
learn = tabular_learner(data, layers=[1000,500], ps=[0.001,0.01], emb_drop=0.04, 
                        y_range=y_range, metrics=exp_rmspe)

In [ ]:
learn.model

In [ ]:
len(data.train_ds.cont_names)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 1e-3, wd=0.2)

In [ ]:
learn.save('1')

In [ ]:
learn.recorder.plot_losses(last=-1)

In [ ]:
learn.load('1');

In [ ]:
learn.fit_one_cycle(5, 3e-4)

In [ ]:
learn.fit_one_cycle(5, 3e-4)

(10th place in the competition was 0.108)

## fin